In [1]:
# #!/usr/bin/python


# | Authors:  | Bex Dunn|
# |----------|----------------|
# | Created: | Jan 7, 2019 |
# | Last edited: | Jan 22,2019 |


#  Before running this script load these modules:
# `module use /g/data/v10/public/modules/modulefiles` 
# `module load dea`
# This code is designed to run on raijin, on the Australian NCI. 
# The shell script to run this code has a *.sh extension


# If you find an error in this code, please raise an issue at https://github.com/GeoscienceAustralia/dea-notebooks
# 
# This code takes a supplied shapefile of a polygon and queries Digital Earth
# Australia http://geoscienceaustralia.github.io/digitalearthau/
# for WOfS, Fractional Cover and NBART. It calculates thresholded tasselled cap wetness. The dominant result for
# each pixel is calculated and the percentage area of the polygon covered by water, wet vegetation, 
# photosynthetic vegetation, non-photosynthetic vegetation and bare soil is output into a jpg stacked plot and to
# csv. The resulting data can be used to monitor changes in wetland behaviour spatiotemporally. 

# - Input Datasets:
# - Landsat 5
# - Landsat 7
# - Landsat 8

# -- Fractional Cover --
# - PV - Photosythetic vegetation
# - NPV - Non-Photosythetic vegetation
# - BS - Bare Soil

# - WOfS Feature Layers (WOFLs)

# __Future Work:__ 
# - do this by max extent of wetness
# - add rainfall for context -- waiting on an update as to availability of data        

### Import Statements: import the modules we need ------------------------------

#$#$#$#$#$ bunch of modules from CK -- may not need

import csv
import multiprocessing
from math import ceil
#$#$#$#$#$

import datacube
import datetime
import fiona
import geopandas as gpd
import numpy as np
import pandas as pd
import rasterio.mask
import rasterio.features
from shapely import geometry
import seaborn as sns
import sys
import time
import xarray as xr

#keep the plotting modules in here as we want to output the stackplots to *.jpg
import matplotlib.dates as mdates
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt

from datacube.storage import masking
from datacube.utils import geometry

#append path to dea notebooks scripts to the system so we can access it
sys.path.append('/g/data/r78/rjd547/jupyter_notebooks/dea-notebooks/10_Scripts')
import DEADataHandling, DEAPlotting, TasseledCapTools

# setup the datacube 
dc = datacube.Datacube(app='asset drill')

### Running into wonderful shapefile complexities here

In [2]:
### Set up polygon
poly_path='/g/data/r78/rjd547/Ramsar_Wetlands/ramsar2015_1.shp'
print(f'Shape file is {poly_path}')

#part = sys.argv[1] ###FIXME: update before running on raijin
part = 1
part = int(part)
print(f'system argument received is {part}')

global Output_dir
Output_dir = '/g/data/r78/rjd547/Ramsar_Wetlands/Ramsar_Outputs_1/'

# add in a delay between dc.load calls to avoid overloading the database - 5 seconds in this case
time.sleep(5*part)
#open the polygon


with fiona.open(poly_path) as shapes:
        crs = geometry.CRS(shapes.crs_wkt)
        #get the list of all the shapes in the shapefile
        ShapesList = list(shapes)
        #Desired number of chunks
        #Set this to 32 because we have 32 CPUs that we'd like to run across
        DesiredChunks = 32
        ChunkSize = ceil(len(ShapesList)/DesiredChunks) #this was set due to Claire having 64000 polygons in her code
        print(f'chunk size is {ChunkSize}')
        print(f'There are {int(len(ShapesList)/ChunkSize)} generated chunks')
        shapessubset = shapes[(part - 1) * ChunkSize: part * ChunkSize]
        print(f'Running for polygon IDs in the range {(part - 1) * ChunkSize} to {part * ChunkSize}')        

Shape file is /g/data/r78/rjd547/Ramsar_Wetlands/ramsar2015_1.shp
system argument received is 1
chunk size is 4
There are 27 generated chunks
Running for polygon IDs in the range 0 to 4


### realise shapes is actually shapessubset now

In [3]:
shapessubset[0]['properties']

OrderedDict([('OBJECTID', 1),
             ('REFCODE', 1),
             ('RAMSAR_NAM', 'Cobourg Peninsula'),
             ('WETLAND_NA', 'Cobourg Peninsula'),
             ('STATE', 'NT'),
             ('JURISDICTI', 'NT'),
             ('DESIGNATIO', '1974-05-08'),
             ('TOTAL_SITE', 226481.0),
             ('GAZETTED_A', 220700.0),
             ('SOURCE', 'State agencies; 1:100 000 to satellite imagery'),
             ('Metadata_U',
              'http://www.environment.gov.au/fed/catalog/main/home.page'),
             ('WetlandsDB',
              'http://www.environment.gov.au/cgi-bin/wetlands/ramsardetails.pl?refcode=1'),
             ('SHAPE_Leng', 6.81510482166),
             ('SHAPE_Area', 0.18756577947),
             ('AREA_HA', 226480.681675)])

In [4]:
attempt_count =0
#     first_geometry = shapes['geometry']
#     polyName = shapes['properties']['ID']
#     #print(polyName)
# #    polyArea = shapes['properties']['area']
#     geom = geometry.Geometry(first_geometry, crs=crs)

#     ## Set up the query, and load in all of the WOFS layers
#     query = {'geopolygon': geom}
#     WOFL = dc.load(product='wofs_albers', **query)

In [5]:
#open the polygon
with fiona.open(poly_path) as shapes: #shapes is a fiona.ogrext.Iterator object and is not subscriptable
        #crs = geometry.CRS(shapes.crs_wkt)
        print(crs)
        first_geometry = shapes['geometry']


PROJCS["GDA94_Australian_Albers",GEOGCS["GCS_GDA_1994",DATUM["Geocentric_Datum_of_Australia_1994",SPHEROID["GRS_1980",6378137,298.257222101]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Albers_Conic_Equal_Area"],PARAMETER["standard_parallel_1",-18],PARAMETER["standard_parallel_2",-36],PARAMETER["latitude_of_center",0],PARAMETER["longitude_of_center",132],PARAMETER["false_easting",0],PARAMETER["false_northing",0],UNIT["Meter",1]]


In [6]:
#open the polygon
with fiona.open(poly_path) as shapes:
        crs = geometry.CRS(shapes.crs_wkt)
        # if our geometry type is a MultiPolygon, which we strongly suspect it is...
        if ((next(iter(shapes))['geometry']['type'])) == 'MultiPolygon':
            print((next(iter(shapes))['geometry']['type']))
            
         

MultiPolygon


In [7]:
for i,layername in enumerate(fiona.listlayers(poly_path)):
    with fiona.open(poly_path, layer=i) as shapes:
        print(i,layername, shapes.values())
        #print(shapes['properties'])

0 ramsar2015_1 <fiona.ogrext.Iterator object at 0x7fa2589f7938>


In [8]:
#plot polygon to check it looks ok
plt.clf()
shape_plot = gpd.read_file(poly_path)
print(shape_plot.WETLAND_NA)
#plt.show()
print(shape_plot.OBJECTID)

0                                      Cobourg Peninsula
1                                   Kakadu National Park
2                                        Moulting Lagoon
3                                           Logan Lagoon
4                                                Lavinia
5                             Pitt Water-Orielton Lagoon
6                                         Apsley Marshes
7                  East Coast Cape Barren Island Lagoons
8                     Flood Plain Lower Ringarooma River
9                                           Jocks Lagoon
10                            Interlaken (Lake Crescent)
11                                Little Waterhouse Lake
12                                          Corner Inlet
13                                         Barmah Forest
14                                       Gunbower Forest
15                                           Lake Arawak
16                                        Lake Bitterang
17                             

<Figure size 432x288 with 0 Axes>

In [ ]:
print(shape_plot.OBJECTID[0])

1


### Set up query

In [ ]:
with fiona.open(poly_path) as shapes:
    crs = geometry.CRS(shapes.crs_wkt)
    first_geometry = next(iter(shapes))['geometry']
    geom = geometry.Geometry(first_geometry, crs=crs)

In [ ]:


query = {'geopolygon': geom,
         #'time': ('2016-01-01', '2018-06-30')
         #'time': ('2007-01-01', '2007-06-30')
         'time': ('1987-01-01', '2019-01-01')
         }

## Set up datasets

### set cloudmasking threshold and load landsat nbart data

In [ ]:
#set cloudmasking threshold and load landsat nbart data
landsat_masked_prop = 0.90
ls578_ds = DEADataHandling.load_clearlandsat(dc=dc, query=query, product='nbart',
        masked_prop=landsat_masked_prop)

Loading ls5 pixel quality
    Loading 160 filtered ls5 timesteps


### mask the data with our original polygon to remove extra data 

In [ ]:
data = ls578_ds
mask = rasterio.features.geometry_mask([geom.to_crs(data.geobox.crs)for geoms in [geom]],
                                           out_shape=data.geobox.shape,
                                           transform=data.geobox.affine,
                                           all_touched=False,
                                           invert=False)

In [ ]:
#for some reason xarray is not playing nicely with our old masking function
mask_xr = xr.DataArray(mask, dims = ('y','x'))
ls578_ds = data.where(mask_xr==False)

In [ ]:
#transform the nbart into tci
tci = TasseledCapTools.thresholded_tasseled_cap(ls578_ds,wetness_threshold=-350, drop=True , drop_tc_bands=True)

### create a masked version of the extent of overthreshold wetness

In [ ]:
#select only finite values (over threshold values)
tcw = xr.ufuncs.isfinite(tci.wetness_thresholded)

In [ ]:
# #reapply the polygon mask
tcw = tcw.where(mask_xr==False)

### load wofls and select only wet pixels

In [ ]:
#load wofs
wofls = dc.load(product = 'wofs_albers', like=ls578_ds)

In [ ]:
#only get wet obs
wetwofl = masking.make_mask(wofls, wet=True)

In [ ]:
#match the wofs observations to the nbart
wetwofl=wetwofl.where(wofls.time==ls578_ds.time)

### mask the wofs obs

In [ ]:
#mask the wofs obs with the polygon mask
wetwofl = wetwofl.where(mask_xr==False)

### load in fractional cover data

In [ ]:
#load the data according to our query
#choose a mask proportion to look for a clear timestep
fc_ds = DEADataHandling.load_clearlandsat(dc, query,product='fc',masked_prop=0.90)

### mask FC with polygon

In [ ]:
fc_ds = fc_ds.where(mask_xr==False)

### mask FC with wetness

In [ ]:
fc_ds_noTCW=fc_ds.where(tcw==False)

In [ ]:
#set scene to plot
scene =2

In [ ]:
#set up our images on a grid using gridspec
plt.clf()
plt.figure(figsize=(12,8))
gs = gridspec.GridSpec(2,2) # set up a 2 x 2 grid of 4 images for better presentation

ax1=plt.subplot(gs[0,0])
fc_ds_noTCW.PV.isel(time=scene).plot(cmap='gist_earth_r')
ax1.set_title('PV')

ax2=plt.subplot(gs[1,0])
fc_ds_noTCW.BS.isel(time=scene).plot(cmap='Oranges')
ax2.set_title('BS')

ax3=plt.subplot(gs[0,1])
fc_ds_noTCW.NPV.isel(time=scene).plot(cmap='copper')
ax3.set_title('NPV')

ax4=plt.subplot(gs[1,1])
fc_ds_noTCW.UE.isel(time=scene).plot(cmap='magma')
ax4.set_title('UE')

plt.tight_layout()
plt.show()

### Calculate number of pixels in area of interest

In [ ]:
#number of pixels in area of interest
pixels = (mask_xr==0).sum(dim=['x','y'])

In [ ]:
mask_xr==0
mask_xr.count(dim=['x','y'])

In [ ]:
#count number of wofs pixels
wofs_pixels = wetwofl.water.sum(dim=['x','y'])

#count percentage of area of wofs
wofs_area_percent = (wofs_pixels/pixels)*100

#count number of tcw pixels
tcw_pixel_count = tcw.sum(dim=['x','y'])

#calculate percentage area wet
tcw_area_percent = (tcw_pixel_count/pixels)*100

#calculate wet not wofs
tcw_less_wofs = tcw_area_percent-wofs_area_percent

### Calculate the dominant fraction for each pixel in Fractional Cover

In [ ]:
#drop data percentage and Unmixing Error
fc_tester = fc_ds_noTCW.drop(['data_perc','UE'])

#following robbi's advice, cast the dataset to a dataarray
maxFC = fc_tester.to_array(dim='variable', name='maxFC')

#turn FC array into integer only as nanargmax doesn't seem to handle floats the way we want it to
FC_int = maxFC.astype('int8')

#use numpy.nanargmax to get the index of the maximum value along the variable dimension
#BSPVNPV=np.nanargmax(FC_int, axis=0)
BSPVNPV=FC_int.argmax(dim='variable')

FC_mask=xr.ufuncs.isfinite(maxFC).all(dim='variable')

# #re-mask with nans to remove no-data
BSPVNPV=BSPVNPV.where(FC_mask)

In [ ]:
#plot the results to check they look roughly like what we are expecting
plt.clf()
plt.imshow(BSPVNPV[2])
plt.colorbar()

In [ ]:
FC_dominant = xr.Dataset({
    'BS': (BSPVNPV==0).where(FC_mask),
    'PV': (BSPVNPV==1).where(FC_mask),
    'NPV': (BSPVNPV==2).where(FC_mask),
})

In [ ]:
FC_count = FC_dominant.sum(dim=['x','y'])

In [ ]:
#Fractional cover pixel count method
#Get number of FC pixels, divide by total number of pixels per polygon

Bare_soil_percent=(FC_count.BS/pixels)*100

Photosynthetic_veg_percent=(FC_count.PV/pixels)*100

NonPhotosynthetic_veg_percent=(FC_count.NPV/pixels)*100

NoData = 100 - wofs_area_percent- tcw_less_wofs - Photosynthetic_veg_percent - NonPhotosynthetic_veg_percent - Bare_soil_percent

In [ ]:
query['time'][0]

In [ ]:
#set up color palette
pal = [sns.xkcd_rgb["cobalt blue"],
       sns.xkcd_rgb["neon blue"],
       sns.xkcd_rgb["grass"],
       sns.xkcd_rgb["beige"],
       sns.xkcd_rgb["brown"]]#,
      #sns.xkcd_rgb["grey"]]

#make a stacked area plot
plt.clf()
plt.figure(figsize = (12,6))
plt.stackplot(wofs_area_percent.time.values, 
              wofs_area_percent, 
              tcw_less_wofs, 
              Photosynthetic_veg_percent, 
              NonPhotosynthetic_veg_percent,
              Bare_soil_percent,
              NoData,
              labels=['open water',
                      'wet',
                      'PV',
                      'NPV',
                      'BS'#,
                      #'NoData'
                     ], colors=pal, alpha = 0.6)

plt.title('Percentage of area WOfS, Wetness, Fractional Cover')

#set axis limits to the min and max
plt.axis(xmin = query['time'][0], xmax = query['time'][1], ymin = 0, ymax = 100)

#set date ticks every year
years = mdates.YearLocator(2)
yearsFmt = mdates.DateFormatter('%Y')
ax = plt.gca()
ax.xaxis.set_major_locator(years)
ax.xaxis.set_major_formatter(yearsFmt)

#add a legend and a tight plot box
plt.legend(loc='upper right')
plt.tight_layout()

#save the figure
#plt.savefig('/g/data/r78/rjd547/SA_mound_springs/Big_Blythe_small_all2.png')#, transparent=True)
plt.show()